<a href="https://colab.research.google.com/github/nastyabekesheva/DataScience/blob/main/labs/lab_3/Income_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [2]:
def download_data():

    #url = "https://archive.ics.uci.edu/ml/machine-learning-databases/census-income-mld/census-income.data.gz"
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    response = requests.get(url)
    if response.status_code != 200:
        raise ValueError

    with open("census-income.csv", 'wb') as file:
        for chunk in response:
            file.write(chunk)

download_data()

In [3]:
columns = '''Age.
Workclass.
Final Weight.
Education.
Education Number of Years.
Marital-status.
Occupation.
Relationship.
Race.
Sex.
Capital-gain.
Capital-loss.
Hours-per-week.
Native-country.
Income'''.replace('\n', '')
columns = columns.split('.')

In [4]:
df = pd.read_csv('census-income.csv', names=columns)
df.head()

,Age,Workclass,Final Weight,Education,Education Number of Years,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
for i in df.columns:
  js = list(df[i].loc[df[i] == ' ?'].index)
  for j in js:
    df[i][j] = np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
df.isnull().values.any()

True

In [7]:
df = df.fillna(method='ffill')
df = df.fillna(method='bfill') 
df = df.fillna(0)
df.isnull().values.any()

False

In [8]:
df['Sex'] = df['Sex'].replace([" Female", " Male"], [0, 1])
df['Income'] = df['Income'].replace([" <=50K", " >50K"], [0, 1])

df = pd.get_dummies(df)

In [9]:
df

,Age,Final Weight,Education Number of Years,Sex,Capital-gain,Capital-loss,Hours-per-week,Income,Workclass_ Federal-gov,Workclass_ Local-gov,...,Native-country_ Portugal,Native-country_ Puerto-Rico,Native-country_ Scotland,Native-country_ South,Native-country_ Taiwan,Native-country_ Thailand,Native-country_ Trinadad&Tobago,Native-country_ United-States,Native-country_ Vietnam,Native-country_ Yugoslavia
0,39,77516,13,1,2174,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,1,0,0,13,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,0,38,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32557,40,154374,9,1,0,0,40,1,0,0,...,0,0,0,0,0,0,0,1,0,0
32558,58,151910,9,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32559,22,201490,9,1,0,0,20,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import QuantileRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [11]:
X = df.drop('Income', axis=1)
y = df['Income']

In [12]:
X

,Age,Final Weight,Education Number of Years,Sex,Capital-gain,Capital-loss,Hours-per-week,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,...,Native-country_ Portugal,Native-country_ Puerto-Rico,Native-country_ Scotland,Native-country_ South,Native-country_ Taiwan,Native-country_ Thailand,Native-country_ Trinadad&Tobago,Native-country_ United-States,Native-country_ Vietnam,Native-country_ Yugoslavia
0,39,77516,13,1,2174,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,1,0,0,13,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,0,38,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32557,40,154374,9,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32558,58,151910,9,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32559,22,201490,9,1,0,0,20,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=42)

In [14]:
lin_reg = LinearRegression().fit(X_train, y_train)

lin_reg.score(X_test, y_test)

0.35871079835216335

In [15]:
xs

NameError: ignored

In [ ]:
sns.regplot(X_test['Capital-gain'], lin_reg.predict(X_test), scatter_kws={'color': 'black'}, line_kws={'color': 'red'})

In [ ]:
log_reg = LogisticRegression().fit(X_train, y_train)

log_reg.score(X_test, y_test)

In [ ]:
sns.regplot(X_test['Capital-gain'], log_reg.predict(X_test), logistic=True, scatter_kws={'color': 'black'}, line_kws={'color': 'red'})

In [ ]:
pa_reg = PassiveAggressiveRegressor(max_iter=100, random_state=0, tol=1e-3).fit(X_train, y_train)

pa_reg.score(X_test, y_test)

In [ ]:
qua_reg = QuantileRegressor(quantile=0.8).fit(X_train, y_train)

qua_reg.score(X_test, y_test)

In [ ]:
pa_reg.predict(X_test)

plt.plot(X_test['Capital-gain'], pa_reg.predict(X_test))

In [ ]:
ax = plt.gca()

ax.plot(X_test['Capital-gain'], rid_reg.coef_[list(X_test.columns).index('Capital-gain')])
ax.set_xscale("log")
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel("alpha")
plt.ylabel("weights")
plt.title("Ridge coefficients as a function of the regularization")
plt.axis("tight")
plt.show()